In [ ]:
!pip install scrapy
!pip install spider3

In [ ]:
import json
import re
import pandas as pd
import scrapy
from pandas.testing import assert_frame_equal # to compare two dataframes

Như bạn đã biết, World Cup là một giải bóng đá lớn nhất thế giới được tổ chức mỗi 4 năm 1 lần. Vì cuối tháng 11 này kỳ World Cup 2022 sẽ diễn ra, do đó ở Lab 1 này, chúng ta sẽ khởi động môn học bằng việc thu thập dữ liệu của các cầu thủ bóng đá. <b>SoFIFA</b> (https://sofifa.com/) là một trang web lưu trữ dữ liệu của các cầu thủ trong trò chơi bóng đá nổi tiếng FIFA 23 mà có các chỉ số phản ánh gần đúng với phong độ của các cầu thủ bóng đá ngoài đời. Trong phần này, nhiệm vụ đầu tiên của bạn là sẽ cần thu thập ID của các cầu thủ. Mình có check "Terms of Service" của SoFIFA thì không thấy có mục nào nói là cấm parse HTML, vì vậy với mục đích học thì chắc là không có vấn đề gì lớn, miễn là chương trình của bạn đừng "hit" trang web quá nhiều lần trong một khoảng thời gian ngắn thì sẽ không có vấn đề gì. 
</br></br>
Công việc cụ thể của bạn ở phần đầu tiên này là sẽ cần viết class <b>collect_player_url()</b> ở bên dưới. Vì dữ liệu lúc sau các bạn cần thu thập sẽ rất lớn nên ở đây, chúng ta sẽ xài một thư viên có tên scrapy để có thể thu thập dữ liệu lớn được nhanh chóng hơn. Về cách sử dụng thư viện này thì bạn có thể tham khảo thêm tại trang web sau: https://docs.scrapy.org/en/latest/intro/tutorial.html.
</br></br>


<h2><b>Tạo một project mới với scrapy</b><h2>

Để sử dụng thư viện scrapy sau khi cài đặt xong, các bạn sẽ gọi câu lệnh như bên dưới để bắt đầu tạo một project mới với scrapy với tên gọi là <b>"fifa_crawler"</b>.

In [ ]:
!scrapy startproject fifa_crawler

In [ ]:
cd fifa_crawler/fifa_crawler

Sau khi tạo xong project với scrapy, vì thu thập dữ liệu với thư viện này không cho phép xài notebook trực tiếp nên các bạn sau khi hoàn thành xong class <b>collect_player_url(scrapy.Spider)</b> ở bên dưới, các bạn cần tạo một file có tên <b>collect_players_urls.py</b> vào đường dẫn sau <b>fifa_crawler/fifa_crawler/spiders/collect_players_urls.py</b>. Ngoài ra, để tránh việc <b>hit</b> trang web quá nhiều lần, thay vì thu thập toàn bộ ID các cầu thủ của thì các bạn sẽ chỉ cần thu thập 720 ID cầu thủ. Để cho dễ dàng, mình đã để sẵn 1 đường dẫn urls chứa định dạng offset (từng trang của các cầu thủ với mỗi trang chứa 60 cầu thủ khác nhau) để các bạn có thể dễ dàng chuyển trang mới trong lúc thu thập.

In [ ]:
import scrapy

class collect_player_url(scrapy.Spider):
    name = 'players_urls'
    def __init__(self):
        self.count = 60
    def start_requests(self):
        urls = ['https://sofifa.com/players?col=oa&sort=desc&offset=0']
        # # YOUR CODE HERE
        # raise NotImplementedError()
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        # YOUR CODE HERE
        # raise NotImplementedError()
        players_url = response.css('tbody > tr')
        for player in players_url:
            id = player.css('td.col-name > a::attr(href)').get().split('/')
            items = {
                "player_url": '/' + id[1] + '/' + id[2]
            }
            yield items

        if self.count < 720:
            next_page = 'https://sofifa.com/players?col=oa&sort=desc&offset=' + str(self.count)
            self.count += 60
            yield response.follow(next_page, callback=self.parse)

Sau khi đã viết xong class trên, và đã tạo file <b>collect_players_urls.py</b> trong đường dẫn đã đề cập, bạn sẽ tiến hành gọi lệnh như bên dưới để thu thập ID của các cầu thủ bóng đá và lưu vào một file có tên <b>players_urls.json</b> mà chứa trong thư mục dataset. Thư mục dataset này bạn không cần tạo mà khi thu thập dữ liệu scrapy sẽ tự động tạo cho bạn.

In [ ]:
!scrapy crawl players_urls -o dataset/players_urls.json

Sau khi đã có danh sách 720 ID của các cầu thủ đã thu thập từ trang web SoFIFA, bạn sẽ tiến hành thu thập dữ liệu cụ thể của từng cầu thủ ứng với các ID này bằng cách hoàn thành class <b>collect_player_info(scrapy.Spider)</b> như bên dưới. Các bạn cũng lưu ý lại là như mình đã đề cập, việc sử dụng scrapy trực tiếp notebook là không được mà chúng ta sẽ cần tạo tiếp một file mới có tên <b>collect_players_info.py</b> vào cùng đường dẫn như file <b>collect_players_urls.py</b>. Ở đây, cũng để cho tiện thì mình cũng để cho các bạn 1 đường dẫn url mới với ID 231747 ứng với ID đầu tiên mà chúng ta đã thu thập ở bước trước. Nhiệm vụ của các bạn là dựa vào url này để tiếp tục hoàn thành việc parse HTML và thu thập các thông tin chi tiết của ID này (chúng ta cũng làm tương tự cho 719 ID còn lại). Về chi tiết tất cả các dữ liệu các bạn cần thu thập thì mình có để sẵn 1 file test có tên <b>players_info.json</b> là file đã được mình thu thập về thông tin chi tiết của 720 cầu thủ cho các bạn.

In [ ]:
df_test = pd.read_json('./players_info.json', encoding='utf-8-sig')
df_test.iloc[0]

In [ ]:
import scrapy
import json


class collect_player_info(scrapy.Spider):
    name = 'players_info'

    def __init__(self):
        try:
            with open('dataset/players_urls.json') as f:
                self.players = json.load(f)
            self.player_count = 1
        except IOError:
            print("File not found")

    def start_requests(self):
        urls = ['https://sofifa.com/player/231747?units=mks']

        # YOUR CODE HERE
        # raise NotImplementedError()
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
        # YOUR CODE HERE
        #   raise NotImplementedError()
        players_info = response.css('body > div.center > .grid > .col.col-12')

        # hàng tuổi, ngày sinh,...
        info = players_info[0].css(
            'div.bp3-card > .info > div.meta.ellipsis::text')[-1]
        index_overall = players_info[0].css(
            'div.bp3-card > .card.spacing > div.block-quarter')
        # profile
        profile = players_info[0].css('div.block-quarter')[4]
        specialities = players_info[0].css('div.block-quarter')[5]
        # teams
        teams = players_info[0].css('div.block-quarter')[6:8]

        # chỉ số cụ thể
        primary_position = response.css(
            'body > div.center > .grid > .col.col-4')
        specific_index = players_info[1].css(
            'div.block-quarter')  # từng mục chỉ số
        name_specific_index = []  # tên thông số cụ thể
        index_specific_index = []  # chỉ số thông số cụ thể
        for i in range(len(specific_index)):
            duplicate_index = specific_index[i].css(
                'div.card > ul.pl > li > span.plus::text').getall()
            duplicate_index += specific_index[i].css(
                'div.card > ul.pl > li > span.minus::text').getall()
            values = specific_index[i].css(
                'div.card > ul.pl > li > span::text').getall()
            values = [i for i in values if i not in duplicate_index] #xóa phần tử chỉ số plus
            
            name_specific_index.append(values[1::2])
            name_specific_index[i] = [
                x.replace(" ", "") for x in name_specific_index[i]]
            index_specific_index.append(values[::2])

        items = {
            "id": profile.css('div.card > ul.pl >.ellipsis::text')[-1].get(),
            "name": players_info[0].css('.info > h1::text').get(),
            "primary_position": primary_position[1].css('ul.pl > li.ellipsis > .pos::text').get(),
            "positions": players_info[0].css('.info > div.meta.ellipsis > span.pos::text').getall(),
            "age": info.get().split(' ')[1][:-4],
            "birth_date": info.get().split(' ')[4][:-1] + '/'+info.get().split(' ')[2][1:] + '/'+info.get().split(' ')[3][:-1],
            "height": int(info.get().split(' ')[5][:-2]),
            "weight": int(info.get().split(' ')[6][:-2]),
            "Overall Rating": int(index_overall.css('span.bp3-tag::text')[0].get()),
            "Potential": int(index_overall.css('span.bp3-tag::text')[1].get()),
            "Value": index_overall.css('div::text')[4].get(),
            "Wage": index_overall.css('div::text')[6].get(),
        }
        #xử lý mục profile
        profile_keys = profile.css(
            'div.card > ul.pl >.ellipsis > label::text').getall()[:-1]
        upper_profile_value = [i for i in profile.css('div.card > ul.pl >.ellipsis::text').getall(
        ) if i != ' '][:-1]
        index_of_upper_profile_value = list(map(int,upper_profile_value[1:]))
        index_of_upper_profile_value.insert(0,upper_profile_value[0])

        profile_values = index_of_upper_profile_value + profile.css('div.card > ul.pl >.ellipsis > span::text').getall()[-4:]
        
        for i in range(len(profile_keys)):
            items[profile_keys[i]] = profile_values[i]
            
        # Xử lý mục teams
        u = {}
        for i in range(len(teams)):
            u[teams.css('div.card > h5 > a::text').getall()[i]] = int(
                teams.css('div.card > ul.ellipsis.pl > li > span.bp3-tag::text')[i].get())
        items["teams"] = u
        
        x = {
            specific_index[0].css('div.card>h5::text').get().lower(): {
                name_specific_index[0][0]: int(index_specific_index[0][0]),
                name_specific_index[0][1]: int(index_specific_index[0][1]),
                name_specific_index[0][2]: int(index_specific_index[0][2]),
                name_specific_index[0][3]: int(index_specific_index[0][3]),
                name_specific_index[0][4]: int(index_specific_index[0][4]),
            },
            specific_index[1].css('div.card>h5::text').get().lower(): {
                name_specific_index[1][0]: int(index_specific_index[1][0]),
                name_specific_index[1][1]: int(index_specific_index[1][1]),
                name_specific_index[1][2]: int(index_specific_index[1][2]),
                name_specific_index[1][3]: int(index_specific_index[1][3]),
                name_specific_index[1][4]: int(index_specific_index[1][4]),
            },
            specific_index[2].css('div.card>h5::text').get().lower(): {
                name_specific_index[2][0]: int(index_specific_index[2][0]),
                name_specific_index[2][1]: int(index_specific_index[2][1]),
                name_specific_index[2][2]: int(index_specific_index[2][2]),
                name_specific_index[2][3]: int(index_specific_index[2][3]),
                name_specific_index[2][4]: int(index_specific_index[2][4]),
            },
            specific_index[3].css('div.card>h5::text').get().lower(): {
                name_specific_index[3][0]: int(index_specific_index[3][0]),
                name_specific_index[3][1]: int(index_specific_index[3][1]),
                name_specific_index[3][2]: int(index_specific_index[3][2]),
                name_specific_index[3][3]: int(index_specific_index[3][3]),
                name_specific_index[3][4]: int(index_specific_index[3][4]),
            },
            specific_index[4].css('div.card>h5::text').get().lower(): {
                name_specific_index[4][0]: int(index_specific_index[4][0]),
                name_specific_index[4][1]: int(index_specific_index[4][1]),
                name_specific_index[4][2]: int(index_specific_index[4][2]),
                name_specific_index[4][3]: int(index_specific_index[4][3]),
                name_specific_index[4][4]: int(index_specific_index[4][4]),
                name_specific_index[4][5]: int(index_specific_index[4][5]),
            },
            specific_index[5].css('div.card>h5::text').get().lower(): {
                name_specific_index[5][0]: int(index_specific_index[5][0]),
                name_specific_index[5][1]: int(index_specific_index[5][1]),
                name_specific_index[5][2]: int(index_specific_index[5][2]),
            },
            specific_index[6].css('div.card>h5::text').get().lower(): {
                name_specific_index[6][0]: int(index_specific_index[6][0]),
                name_specific_index[6][1]: int(index_specific_index[6][1]),
                name_specific_index[6][2]: int(index_specific_index[6][2]),
                name_specific_index[6][3]: int(index_specific_index[6][3]),
                name_specific_index[6][4]: int(index_specific_index[6][4]),
            },
            "player_traits":
                specific_index[7].css(
                    'div.card > ul.pl > li > span::text').getall(),
            "player_specialities": specialities.css('div.card > ul.pl >.ellipsis > a::text').getall()
        }
        items.update(x)
        yield items

        if self.player_count < len(self.players):
            next_page_url = 'https://sofifa.com' + \
                self.players[self.player_count]['player_url'] + '?units=mks'
            self.player_count += 1
            yield scrapy.Request(url=next_page_url, callback=self.parse)



Sau khi đã hoàn thành class ở trên và lưu lại trong file <b>collect_players_info.py</b>, các bạn sẽ tiếp tục chạy câu lệnh bên dưới để thu thập thông tin chi tiết của toàn bộ 720 cầu thủ và xuất ra file <b>players_info.json</b> ở cùng đường dẫn dataset như trên.

In [ ]:
!scrapy crawl players_info -o dataset/players_info.json

Sau khi đã có thông tin chi tiết của 720 cầu thủ, các bạn sẽ tiến hành đọc file <b>players_info.json</b> vào pandas với tên gọi <b>df_players_info</b> và kiểm tra lại với file của mình xem dữ liệu các bạn thu thập là khớp chưa. Nếu đã trùng khớp hai file thì bạn sẽ đến với bước tiền xử lý dữ liệu.

In [ ]:
# YOUR CODE HERE
# raise NotImplementedError()
df_players_info = pd.read_json('./fifa_crawler/fifa_crawler/dataset/players_info.json', encoding='utf-8-sig')
assert_frame_equal(df_players_info, df_test)

<h2><b>Tiền xử lý dữ liệu</b></h2>

<b>1)</b> Sau khi đã kiểm tra và khớp với dữ liệu từ file mình đã cung cấp, nhiệm vụ đầu tiên của các bạn trong bước tiền xử lý dữ liệu là cần phân tách các chỉ số cụ thể của mỗi cầu thủ. Như chúng ta thấy, ở mỗi cột <b>attacking, skill, movement</b> sẽ là một từ điển chứa chi tiết cụ thể từng chỉ số chi tiết ở trong đó. Do đó, chúng ta sẽ cần phân tách các từ điển này thành các cột cụ thể và nhiệm vụ khi này của bạn là hoàn thành hàm <b>split_players_info()</b> ở bên dưới rồi lưu lại vào một dataframe mới với tên <b>df_split_players_info</b>.

In [ ]:
def split_players_info():
    # YOUR CODE HERE
    # raise NotImplementedError()
    # lấy tên cột attacking, skill,...
    names_skill = df_players_info.columns[-9:-2]

    skill_keys = []  # lưu tên các thuộc tính của mỗi cột
    for i in names_skill:
        skill_keys += df_players_info[i][0].keys()

    skill_dict = {}
    for index in skill_keys:
        skill_dict[index] = {}

    for i in range(len(df_players_info)):
        for element in names_skill:
            for index in skill_keys:
                if index in df_players_info[element][0].keys():
                    skill_dict[index][i] = df_players_info[element][i][index]

    df_split_players_info = pd.DataFrame(skill_dict)
    df_split_players_info = pd.concat([df_players_info, df_split_players_info], axis=1)
    for i in names_skill:
        del df_split_players_info[i]

    return df_split_players_info


In [ ]:
# TEST
df_split_players_info = split_players_info()
df_split_players_info_test = pd.read_json('./split_players_info.json', encoding='utf-8-sig')
assert_frame_equal(df_split_players_info.iloc[:, 23:], df_split_players_info_test.iloc[:, 23:])

<b>2)</b> Vì cột giá trị của cầu thủ (Value) và tiền lương (Wage) đang ở dạng viết tắt nên các bạn cần chuyển hai cột này về dạng số (float) bằng cách hoàn thành hàm <b>value_and_wage_to_float(col)</b>. Ví dụ, cầu thủ A có giá trị là €1M và tiền lương là €1K, hai cột này sau khi được tiền xử lý thì cầu thủ A sẽ có giá trị là 1000000.0 và tiền lương là 1000. Ngoài ra, bạn cần lưu ý kiểm tra các cột này có giá trị bị thiếu hay không, nếu có sẽ điền là 0.0 và ngược lại sẽ tiền xử lý như trên.

In [ ]:
def value_and_wage_to_float(col):
    # YOUR CODE HERE
    # raise NotImplementedError()
    for i in range(len(df_split_players_info)):
        if type(df_split_players_info[col][i]) == str:
            temp = df_split_players_info[col][i][1:-1]
            unit = df_split_players_info[col][i][-1]

            if temp =='':
                temp = '0'
            if unit == 'M':
                temp = float(temp) * 1000000
            elif unit == 'K':
                temp = float(temp) * 1000
            df_split_players_info[col][i] = float(temp)

In [ ]:
value_and_wage_to_float('Value')
value_and_wage_to_float('Wage')

In [ ]:
# TEST
assert df_split_players_info['Wage'].to_list()[:5] == [230000.0, 350000.0, 420000.0, 450000.0, 195000.0]
assert df_split_players_info['Wage'].to_list()[-5:] == [41000.0, 21000.0, 59000.0, 15000.0, 50000.0]
assert df_split_players_info['Value'].to_list()[:5] == [190500000.0, 107500000.0, 84000000.0, 64000000.0, 54000000.0]
assert df_split_players_info['Value'].to_list()[-5:] == [32000000.0, 9500000.0, 19500000.0, 22000000.0, 20000000.0]

<b>3)</b> Vì cột giá trị giải phóng hợp đồng (Release Clause) của cầu thủ đang ở dạng object, các bạn cần chuyển về dạng chuỗi và sau đó tiến hành tiền xử lý tương tự như hai cột <b>Value</b> và <b>Wage</b> ở trên bằng cách hoàn thành hàm <b>release_clause_to_float(col)</b>.

In [ ]:
import numpy as np

def release_clause_to_float(col):
    # YOUR CODE HERE
    # raise NotImplementedError()
    for i in range(len(df_split_players_info)):
        if type(df_split_players_info[col][i]) == str:
            temp = df_split_players_info[col][i][1:-1]
            unit = df_split_players_info[col][i][-1]

            if temp == '':
                temp = '0'
            if unit == 'M':
                temp = float(temp) * 1000000
            elif unit == 'K':
                temp = float(temp) * 100000
            df_split_players_info[col][i] = float(temp)
    df_split_players_info[col] = df_split_players_info[col].fillna(0)
    

In [ ]:
release_clause_to_float('Release Clause')

In [ ]:
# TEST
assert df_split_players_info['Release Clause'].to_list()[:5] == [366700000.0, 198900000.0, 172200000.0, 131199999.99999999, 99900000.0]
assert df_split_players_info['Release Clause'].to_list()[-5:] == [0.0, 17100000.0, 38500000.0, 48400000.0, 35500000.0]